In [3]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
# pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


In [4]:
# Data manipulation
import pandas as pd
import numpy as np
import sklearn
import asyncio
import requests
import re

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Visualizations
import matplotlib.pyplot as plt

In [5]:
import requests
_site_URL = "http://envisoft.cem.gov.vn/eos/services/call/json/get_stations"

r = requests.post(url=_site_URL)
site_data = r.json()
site_data = pd.json_normalize(site_data['stations'])
site_data = site_data[['off_time3', "station_name", "address", "id", 'latitude', 'longitude']]
site_data.rename(columns={'id': 'dataid', 'longitude': 'longtitude'}, inplace=True)
site_data['longtitude'] = site_data['longtitude'].round(4)
site_data['latitude'] = site_data['latitude'].round(4)
site_data = site_data[site_data['station_name'].str.contains("Hà Nội:")]
# site_data
site_data_path = os.path.join(PROJ_ROOT,
                              "data",
                              "raw",
                              "site_data",
                              "site_data_cem_2.csv")
site_data[['dataid', 'station_name', 'off_time3', 'address', 'latitude', 'longtitude']].to_csv(site_data_path, index=False)

In [9]:
site_data_path = os.path.join(PROJ_ROOT,
                              "data",
                              "raw",
                              "site_data",
                              "site_data_cem_2.csv")
site_data = pd.read_csv(site_data_path, dtype={"dataid":"string"})
site_data

,id,dataid,station_name,off_time3,address,latitude,longtitude
0,10,29195888808520421980665944934,Hà Nội: 36A Phạm Văn Đồng - Trạm cảm biến (KK),NaN,18 Huỳnh Thúc Kháng - Đống Đa - Hà Nội,21.0512,105.7823
1,2,28560877461938780203765592307,Hà Nội: 556 Nguyễn Văn Cừ (KK),NaN,"Nguyễn Văn Cừ, Long Biên",21.0491,105.8831
2,14,29195924558310436897556929382,Hà Nội: Chi cục BVMT (KK),NaN,18 Huỳnh Thúc Kháng - Đống Đa - Hà Nội,21.0153,105.8001
3,33,29573032472660669416148855365,Hà Nội: Cung thiếu nhi Hà Nội - Hoàn Kiếm - T...,4 ngày trước,36 Lý Thái Tổ,21.0285,105.8554
4,16,29195931346712256033392565094,Hà Nội: Đại sứ quán Pháp (KK),1 year ago,18 Huỳnh Thúc Kháng - Đống Đa - Hà Nội,21.0213,105.8509
...,...,...,...,...,...,...,...
35,47,29573151970668779242856409669,"Hà Nội: UBND TT Sóc Sơn, huyện Sóc Sơn - Trạm ...",4 ngày trước,"UBND thị trấn Sóc Sơn, huyện Sóc Sơn",21.2574,105.8511
36,45,29573138947267463069477136965,Hà Nội: UBND TT Xuân Mai - Chương Mỹ - Trạm cả...,4 ngày trước,UBND TT Xuân Mai - Chương Mỹ,20.8993,105.5773
37,38,29573062559300253892212976197,Hà Nội: UBND xã An Khánh - Hoài Đức - Trạm cảm...,5 tháng trước,UBND xã An Khánh - Hoài Đức,21.0024,105.7180
38,46,29571660496070187292943477317,Hà Nội: UBND xã Thanh Xuân - Sóc Sơn - Trạm cả...,4 ngày trước,UBND xã Thanh Xuân - Sóc Sơn,21.2288,105.7579


In [7]:
headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0',
  'Accept': 'application/json, text/javascript, */*; q=0.01',
  'Accept-Language': 'en-US,en;q=0.5',
  'Referer': 'http://enviinfo.cem.gov.vn/',
  'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
  'X-Requested-With': 'XMLHttpRequest',
  'Origin': 'http://enviinfo.cem.gov.vn',
  'Connection': 'keep-alive',
  'Cookie': 'session_id_eip=125.235.239.3-7aa0263d-aede-4574-94e8-19e09900c1aa',
  'Pragma': 'no-cache',
  'Cache-Control': 'no-cache'
}
url = "http://enviinfo.cem.gov.vn/eip/default/call/json/get_aqi_data%3Faqi_type%3D1"
    
payload = "sEcho=1&iColumns=7&sColumns=%2C%2C%2C%2C%2C%2C&iDisplayStart=0&iDisplayLength=40000&mDataProp_0=0&sSearch_0=&bRegex_0=false&bSearchable_0=true&mDataProp_1=1&sSearch_1=&bRegex_1=false&bSearchable_1=true&mDataProp_2=2&sSearch_2=&bRegex_2=false&bSearchable_2=true&mDataProp_3=3&sSearch_3=&bRegex_3=false&bSearchable_3=true&mDataProp_4=4&sSearch_4=&bRegex_4=false&bSearchable_4=true&mDataProp_5=5&sSearch_5=&bRegex_5=false&bSearchable_5=true&mDataProp_6=6&sSearch_6=&bRegex_6=false&bSearchable_6=true&sSearch=&bRegex=false&station_id={}&added_columns=PM-10%2CPM-2-5".format('29195931346712256033392565094')
df = pd.DataFrame()
# columns[0:0] = [0, 'time', 'AQI']
response = requests.request("POST", url, headers=headers, data=payload)
data = response.json()
data


{'iTotalRecords': 573,
 'aoColumns': [],
 'aaData': [['1', '25/03/2020 20:00', '28', '28'],
  ['2', '25/03/2020 19:00', '32', '32'],
  ['3', '25/03/2020 18:00', '35', '35'],
  ['4', '25/03/2020 17:00', '38', '38'],
  ['5', '25/03/2020 16:00', '34', '34'],
  ['6', '25/03/2020 15:00', '31', '31'],
  ['7', '25/03/2020 14:00', '27', '27'],
  ['8', '25/03/2020 13:00', '32', '32'],
  ['9', '25/03/2020 12:00', '35', '35'],
  ['10', '25/03/2020 11:00', '37', '37'],
  ['11', '25/03/2020 10:00', '31', '31'],
  ['12', '25/03/2020 09:00', '29', '29'],
  ['13', '25/03/2020 08:00', '32', '32'],
  ['14', '25/03/2020 07:00', '34', '34'],
  ['15', '25/03/2020 06:00', '42', '42'],
  ['16', '25/03/2020 05:00', '49', '49'],
  ['17', '25/03/2020 04:00', '53', '53'],
  ['18', '25/03/2020 03:00', '50', '50'],
  ['19', '25/03/2020 02:00', '51', '51'],
  ['20', '25/03/2020 01:00', '46', '46'],
  ['21', '25/03/2020 00:00', '46', '46'],
  ['22', '24/03/2020 23:00', '52', '52'],
  ['23', '24/03/2020 22:00', '50',

In [19]:
def get_columns(headers, dataid):
    url = "http://enviinfo.cem.gov.vn/eip/default/call/json/get_indicators_have_data"
    payload = "station_id={}&from_public=0&station_type=4".format(dataid)
    response = requests.request("POST", url, headers=headers, data=payload)

    response = response.json()

    response = response['html']

    response = response.replace("PM-2-5", "PM2.5")
    response = response.replace("PM-10", "PM10")
    # x = re.findall("selected>(.*?)</option>", response)
    return payload
a = get_columns(headers=headers, dataid="29195931346712256033392565094")
a

'station_id=29195931346712256033392565094&from_public=0&station_type=4'

In [21]:
def get_raw_aqi_data(headers, dataid, id, columns):
    url = "http://enviinfo.cem.gov.vn/eip/default/call/json/get_aqi_data%3Faqi_type%3D1"
    
    payload = "sEcho=1&iColumns=7&sColumns=%2C%2C%2C%2C%2C%2C&iDisplayStart=0&iDisplayLength=40000&mDataProp_0=0&sSearch_0=&bRegex_0=false&bSearchable_0=true&mDataProp_1=1&sSearch_1=&bRegex_1=false&bSearchable_1=true&mDataProp_2=2&sSearch_2=&bRegex_2=false&bSearchable_2=true&mDataProp_3=3&sSearch_3=&bRegex_3=false&bSearchable_3=true&mDataProp_4=4&sSearch_4=&bRegex_4=false&bSearchable_4=true&mDataProp_5=5&sSearch_5=&bRegex_5=false&bSearchable_5=true&mDataProp_6=6&sSearch_6=&bRegex_6=false&bSearchable_6=true&sSearch=&bRegex=false&station_id={}&added_columns=PM-10%2CPM-2-5".format(dataid)
    df = pd.DataFrame()
    try:
        columns[0:0] = [0, 'time', 'AQI']
        response = requests.request("POST", url, headers=headers, data=payload)
        data = response.json()
        data = data['aaData']
        df = pd.DataFrame(data, columns=columns)
        df.drop([0], axis=1, inplace=True)
        df['site_id'] = id
        column_order = ['site_id', 'time', 'CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2', 'AQI']
        df = df.reindex(column_order, axis='columns', fill_value=0)
        df = df[column_order]
        df = df.iloc[::-1]
    except Exception as e:
        # print("error in site {}".format(id))
        # print("columns: {}".format(columns))
        print(data)
        # columns = [0, 'time', 'AQI']
        # response = requests.request("POST", url, headers=headers, data=payload)
        # data = response.json()
        # data = data['aaData']
        # df = pd.DataFrame(data, columns=columns)
        # df.drop([0], axis=1, inplace=True)
        # df['site_id'] = id
        # column_order = ['site_id', 'time', 'CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2', 'AQI']
        # df = df.reindex(column_order, axis='columns', fill_value=0)
        # df = df[column_order]
        # df = df.iloc[::-1]
    return df

def get_columns(headers, dataid):
    url = "http://enviinfo.cem.gov.vn/eip/default/call/json/get_indicators_have_data"
    payload = "station_id={}&from_public=1&station_type=4".format(dataid)
    response = requests.request("POST", url, headers=headers, data=payload)

    response = response.json()

    response = response['html']

    response = response.replace("PM-2-5", "PM2.5")
    response = response.replace("PM-10", "PM10")
    x = re.findall("selected>(.*?)</option>", response)
    return x

headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:92.0) Gecko/20100101 Firefox/92.0',
  'Accept': 'application/json, text/javascript, */*; q=0.01',
  'Accept-Language': 'en-US,en;q=0.5',
  'Referer': 'http://enviinfo.cem.gov.vn/',
  'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
  'X-Requested-With': 'XMLHttpRequest',
  'Origin': 'http://enviinfo.cem.gov.vn',
  'Connection': 'keep-alive',
  'Cookie': 'session_id_eip=125.235.239.3-7aa0263d-aede-4574-94e8-19e09900c1aa',
  'Pragma': 'no-cache',
  'Cache-Control': 'no-cache'
}

# get_raw_aqi_data(site_data['dataid'][1])
for id, dataid in zip(site_data['id'], site_data['dataid']):
    # dataid = data['dataid']
    print("{} --- {}".format(id,dataid))
    columns = get_columns(headers=headers, dataid=dataid)
    response_data = get_raw_aqi_data(dataid=str(dataid), id=id, headers=headers, columns=columns)
    if not response_data.empty:
        cem_site_path = os.path.join(PROJ_ROOT,
                                "data",
                                "cem",
                                "{}.csv".format(id))
        response_data.to_csv(cem_site_path, index=False)


10 --- 29195888808520421980665944934
2 --- 28560877461938780203765592307
14 --- 29195924558310436897556929382
33 --- 29573032472660669416148855365
16 --- 29195931346712256033392565094
error in site 16
columns: [0, 'time', 'AQI']
[['1', '25/03/2020 20:00', '28', '28'], ['2', '25/03/2020 19:00', '32', '32'], ['3', '25/03/2020 18:00', '35', '35'], ['4', '25/03/2020 17:00', '38', '38'], ['5', '25/03/2020 16:00', '34', '34'], ['6', '25/03/2020 15:00', '31', '31'], ['7', '25/03/2020 14:00', '27', '27'], ['8', '25/03/2020 13:00', '32', '32'], ['9', '25/03/2020 12:00', '35', '35'], ['10', '25/03/2020 11:00', '37', '37'], ['11', '25/03/2020 10:00', '31', '31'], ['12', '25/03/2020 09:00', '29', '29'], ['13', '25/03/2020 08:00', '32', '32'], ['14', '25/03/2020 07:00', '34', '34'], ['15', '25/03/2020 06:00', '42', '42'], ['16', '25/03/2020 05:00', '49', '49'], ['17', '25/03/2020 04:00', '53', '53'], ['18', '25/03/2020 03:00', '50', '50'], ['19', '25/03/2020 02:00', '51', '51'], ['20', '25/03/2020 